# electricity_market_player

> This module training optimizing and evaluation of RL agent on the electricity market environment.
using PPO with actions mask.

In [ ]:
# | default_exp electricity_market_player

In [ ]:
# | hide
from nbdev.showdoc import *

In [ ]:
# | export
from abc import ABC
from collections import defaultdict
from dataclasses import dataclass
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import optuna
import rliable
import seaborn as sns
import torch
import yaml
from sb3_contrib import MaskablePPO
from sb3_contrib.common.maskable.policies import MaskableActorCriticPolicy
from sb3_contrib.common.wrappers import ActionMasker
from stable_baselines3 import A2C
from stable_baselines3.common.callbacks import CheckpointCallback
from stable_baselines3.common.monitor import Monitor
from tqdm.notebook import tqdm

from electricity_market.electricity_market_env import ElectricityMarketEnv, EnvConfig

In [ ]:
# | export
N_EPISODES = 3
N_TRAILS = 10
SEEDS = [
    100000,
    101010,
    101055,
    111111,
    123456,
    157724,
    200000,
    216555,
    217890,
    222222,
    224775,
    234567,
    253084,
    285234,
    312135,
    314831,
    333333,
    345678,
    406339,
    444444,
    471678,
    526070,
    526785,
    555555,
    562845,
    666666,
    701753,
    730845,
    755460,
    761386,
    777777,
    789391,
    877838,
    878787,
    888888,
    900000,
    993068,
    979797,
    987654,
    989898,
    999999,
]


training_data_per_agent = {}
evaluation_data_per_agent = {}

In [ ]:
# | export


@dataclass
class TrainingData:
    steps: list[int]
    episodes: list[int]
    rewards: list[float]


@dataclass
class EvaluationData:
    episodes: list[int]
    rewards: list[float]

In [ ]:
# | export


def plot_all_metrics(
    agent_train_data: dict[str, TrainingData],
    agent_eval_data: dict[str, EvaluationData],
):
    sns.set_theme(style="whitegrid")

    def plot_learning_curve():
        for agent, data in agent_train_data.items():
            plt.plot(data.steps, data.rewards, label=f"{agent} Learning Curve")
        plt.xlabel("Steps")
        plt.ylabel("Reward")
        plt.title("Learning Curves")
        plt.legend()
        plt.show()

    def plot_training_stability():
        for agent, data in agent_train_data.items():
            plt.plot(
                data.steps,
                np.cumsum(data.rewards) / (np.arange(len(data.steps)) + 1),
                label=f"{agent} Stability",
            )
        plt.xlabel("Steps")
        plt.ylabel("Cumulative Average Reward")
        plt.title("Training Stability")
        plt.legend()
        plt.show()

    def plot_sample_efficiency():
        for agent, data in agent_train_data.items():
            plt.plot(
                data.steps,
                np.cumsum(data.rewards) / (np.arange(len(data.steps)) + 1),
                label=f"{agent} Sample Efficiency",
            )
        plt.xlabel("Steps")
        plt.ylabel("Cumulative Average Reward")
        plt.title("Sample Efficiency Curve")
        plt.legend()
        plt.show()

    def plot_time_to_convergence():
        for agent, data in agent_train_data.items():
            # Ensure that data.rewards is a 1D array before using np.diff
            if len(np.shape(data.rewards)) == 1:
                converged_step = np.argmax(
                    np.diff(data.rewards) < 0.01
                )  # Example threshold for convergence
                plt.axvline(
                    x=data.steps[converged_step], label=f"{agent} Time to Convergence"
                )
            else:
                print(f"Skipping {agent} due to invalid rewards data shape.")
        plt.xlabel("Steps")
        plt.ylabel("Reward")
        plt.title("Time-to-Convergence")
        plt.legend()
        plt.show()

    def plot_aggregate_metrics():
        for agent, data in agent_eval_data.items():
            plt.plot(data.episodes, data.rewards, label=f"{agent} Performance")
        plt.xlabel("Episodes")
        plt.ylabel("Reward")
        plt.title("Aggregate Evaluation Metrics")
        plt.legend()
        plt.show()

    def plot_probability_of_improvement():
        improvements = defaultdict(list)
        for agent, data in agent_eval_data.items():
            baseline_rewards = data.rewards[: len(data.rewards) // 2]
            improvement = np.mean(data.rewards[len(data.rewards) // 2 :]) - np.mean(
                baseline_rewards
            )
            improvements[agent] = improvement
        plt.bar(improvements.keys(), improvements.values())
        plt.xlabel("Agent")
        plt.ylabel("Average Improvement")
        plt.title("Probability of Improvement Between Algorithms")
        plt.show()

    def plot_performance_profiles():
        all_rewards = defaultdict(list)
        for agent, data in agent_eval_data.items():
            all_rewards[agent] = data.rewards
        for agent, rewards in all_rewards.items():
            sorted_rewards = np.sort(rewards)
            plt.plot(
                np.arange(len(sorted_rewards)),
                sorted_rewards,
                label=f"{agent} Performance Profile",
            )
        plt.xlabel("Episodes")
        plt.ylabel("Reward")
        plt.title("Performance Profiles")
        plt.legend()
        plt.show()

    def plot_catastrophic_forgetting():
        for agent, data in agent_eval_data.items():
            forgetting = np.abs(
                np.array(data.rewards) - np.mean(data.rewards)
            )  # Simplified measure
            plt.plot(data.episodes, forgetting, label=f"{agent} Forgetting")
        plt.xlabel("Episodes")
        plt.ylabel("Catastrophic Forgetting (Deviation from Mean Reward)")
        plt.title("Catastrophic Forgetting")
        plt.legend()
        plt.show()

    def plot_regret_analysis():
        for agent, data in agent_eval_data.items():
            regret = np.max(data.rewards) - np.array(data.rewards)
            plt.plot(data.episodes, regret, label=f"{agent} Regret")
        plt.xlabel("Episodes")
        plt.ylabel("Regret (Max - Current Reward)")
        plt.title("Regret Analysis")
        plt.legend()
        plt.show()

    def plot_robustness_to_perturbations():
        for agent, data in agent_eval_data.items():
            noise = np.random.normal(
                0, 0.1, size=len(data.rewards)
            )  # Adding noise as perturbation
            robustness = data.rewards + noise
            plt.plot(data.episodes, robustness, label=f"{agent} Robustness")
        plt.xlabel("Episodes")
        plt.ylabel("Reward (with Perturbations)")
        plt.title("Robustness to Perturbations")
        plt.legend()
        plt.show()

    def plot_pareto_frontier():
        all_rewards = defaultdict(list)
        for agent, data in agent_eval_data.items():
            all_rewards[agent] = data.rewards
        for agent, rewards in all_rewards.items():
            plt.scatter(
                np.arange(len(rewards)), rewards, label=f"{agent} Pareto Frontier"
            )
        plt.xlabel("Episodes")
        plt.ylabel("Reward")
        plt.title("Pareto Frontier for Multi-Objective Optimization")
        plt.legend()
        plt.show()

    # Call all the plot functions
    plot_learning_curve()
    plot_training_stability()
    plot_sample_efficiency()
    plot_time_to_convergence()
    plot_aggregate_metrics()
    plot_probability_of_improvement()
    plot_performance_profiles()
    plot_catastrophic_forgetting()
    plot_regret_analysis()
    plot_robustness_to_perturbations()
    plot_pareto_frontier()

In [ ]:
# | export


class Agent(ABC):
    def __init__(self, agent_name):
        self.agent_name = agent_name

    def train(self) -> TrainingData:
        """
        Train the model, and return TrainingData
        """
        raise NotImplementedError

    def evaluate(self, render: bool = False) -> EvaluationData:
        """
        Evaluate the model, and return EvaluationData
        """
        raise NotImplementedError

    def save_model(self, model_path: Path):
        """
        save the model
        """
        raise NotImplementedError

    def load_model(self, model_path: Path) -> None:
        """
        load the model
        """
        raise NotImplementedError


class MaskableAgent(Agent):
    @staticmethod
    def mask_fn(env):
        """
        Placeholder mask function if needed.
        """
        return env.action_masks()

In [ ]:
# | export


class MaskableRandomAgent(MaskableAgent):
    def __init__(
        self, env_config: EnvConfig | None = None, render_mode: str | None = None
    ):
        """
        Initialize the agent and create the environment.
        """
        super().__init__(agent_name="MaskableRandomAgent")
        self.env = ActionMasker(
            ElectricityMarketEnv(env_config, render_mode=render_mode), self.mask_fn
        )

    def train(self) -> TrainingData:
        """
        This agent does not train
        """
        raise NotImplementedError

    def evaluate(self, render: bool = False) -> EvaluationData:
        """
        Evaluates the random agent by executing episodes in the environment.
        """
        all_rewards = []

        for seed in tqdm(SEEDS, desc="seeds"):
            # TODO: utilize sb3-contrib evaluate_policy
            for _ in tqdm(range(N_EPISODES), desc="episodes", leave=False):
                episode_rewards = []

                obs, _ = self.env.reset(seed=seed)
                done = False
                total_reward = 0.0

                while not done:
                    action_mask = self.env.action_masks()
                    valid_actions = np.where(action_mask)[0]  # Get valid actions
                    action = np.random.choice(
                        valid_actions
                    )  # Select random valid action

                    obs, reward, done, truncated, _ = self.env.step(action)
                    total_reward += reward

                    if render:
                        self.env.render()

                    if truncated:
                        break

                episode_rewards.append(total_reward)

                # Append episode results to the total list
                all_rewards.append(total_reward)

        return EvaluationData(
            episodes=list(range(len(all_rewards))),
            rewards=all_rewards,
        )

    def save_model(self, model_path: Path) -> None:
        """
        No model to save for a random agent.
        """
        raise NotImplementedError

    def load_model(self, model_path: Path) -> None:
        """
        No model to load for a random agent.
        """
        raise NotImplementedError

In [ ]:
# | export


class A2CAgent(Agent):
    """A2C Agent for the Electricity Market Environment."""

    def __init__(
        self, env_config: EnvConfig | None = None, render_mode: str | None = None
    ):
        super().__init__(agent_name="A2CAgent")
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.env = Monitor(
            ElectricityMarketEnv(env_config, render_mode=render_mode),
        )
        self.model = A2C(
            "MlpPolicy",
            self.env,
            verbose=0,
            tensorboard_log=f"./{self.agent_name}_tensorboard/",
            device=self.device,
        )

    def train(self) -> TrainingData:
        """
        Train the model, and return TrainingData.
        """
        all_rewards = []
        all_steps = []
        total_steps = 0
        checkpoint_callback = CheckpointCallback(save_freq=1000, save_path="./logs/")

        # Training loop
        for seed in tqdm(SEEDS, desc="seeds"):
            for _ in tqdm(range(N_EPISODES), desc="Training episodes"):
                obs, _ = self.env.reset(seed=seed)
                episode_rewards = []
                steps = 0
                done = False

                while not done:
                    # Convert observation to torch tensor
                    obs_tensor = torch.tensor(obs, dtype=torch.float32).to(self.device)

                    # Get action and action probability from the policy
                    action, _ = self.model.predict(obs_tensor)

                    obs, reward, done, truncated, _ = self.env.step(action)
                    episode_rewards.append(reward)
                    steps += 1

                    # Perform learning step after each episode
                    if done or truncated:
                        self.model.learn(
                            total_timesteps=steps, callback=checkpoint_callback
                        )

                all_rewards.append(np.sum(episode_rewards))
                total_steps += steps
                all_steps.append(total_steps)

        return TrainingData(
            steps=all_steps,
            episodes=list(range(len(all_rewards))),
            rewards=all_rewards,
        )

    def evaluate(self, render: bool = False) -> EvaluationData:
        """
        Evaluate the model, and return EvaluationData.
        """
        all_rewards = []

        for seed in tqdm(SEEDS, desc="seeds"):
            # TODO: utilize sb3 evaluate_policy
            for _ in tqdm(range(N_EPISODES), desc="Evaluation episodes"):
                obs, _ = self.env.reset(seed=seed)
                episode_rewards = []
                done = False

                while not done:
                    # Convert observation to torch tensor
                    obs_tensor = torch.tensor(obs, dtype=torch.float32).to(self.device)

                    action, _ = self.model.predict(obs_tensor, deterministic=True)
                    obs, reward, done, truncated, _ = self.env.step(action)
                    episode_rewards.append(reward)

                    if render:
                        self.env.render()

                    if truncated:
                        break

                all_rewards.append(np.sum(episode_rewards))

        return EvaluationData(
            episodes=list(range(len(all_rewards))),
            rewards=all_rewards,
        )

    def save_model(self, model_path: Path) -> None:
        self.model.save(str(model_path))

    def load_model(self, model_path: Path) -> None:
        self.model.load(str(model_path))

In [ ]:
# | export


class MaskablePPOAgent(MaskableAgent):
    """Maskable PPO Agent for the Electricity Market Environment."""

    def __init__(
        self, env_config: EnvConfig | None = None, render_mode: str | None = None
    ):
        super().__init__(agent_name="MaskablePPOAgent")
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.env_config = env_config or EnvConfig()
        self.env = Monitor(
            ActionMasker(
                ElectricityMarketEnv(env_config, render_mode=render_mode),
                self.mask_fn,
            )
        )
        self.optimized_hyperparameters = {}
        self.model = MaskablePPO(
            MaskableActorCriticPolicy,
            self.env,
            **self.optimized_hyperparameters,
            verbose=0,
            tensorboard_log=f"./{self.agent_name}_tensorboard/",
            device=self.device,
        )

    def train(self) -> TrainingData:
        """
        Train the model, and return TrainingData.
        """
        all_rewards = []
        all_steps = []
        total_steps = 0
        checkpoint_callback = CheckpointCallback(save_freq=1000, save_path="./logs/")

        # Training loop
        for seed in tqdm(SEEDS, desc="seeds"):
            for _ in tqdm(range(N_EPISODES), desc="Training episodes"):
                obs, _ = self.env.reset(seed=seed)
                episode_rewards = []
                steps = 0
                done = False

                while not done:
                    # Convert observation to torch tensor
                    obs_tensor = torch.tensor(obs, dtype=torch.float32).to(self.device)

                    action, _ = self.model.predict(obs_tensor)

                    obs, reward, done, truncated, _ = self.env.step(action)
                    episode_rewards.append(reward)
                    steps += 1

                    # Perform learning step after each episode
                    if done or truncated:
                        self.model.learn(
                            total_timesteps=steps,
                            use_masking=True,
                            callback=checkpoint_callback,
                        )

                all_rewards.append(np.sum(episode_rewards))
                total_steps += steps
                all_steps.append(total_steps)

        return TrainingData(
            steps=all_steps,
            episodes=list(range(len(all_rewards))),
            rewards=all_rewards,
        )

    def evaluate(self, render: bool = False) -> EvaluationData:
        """
        Evaluate the model, and return EvaluationData.
        """
        all_rewards = []

        for seed in tqdm(SEEDS, desc="seeds"):
            for _ in tqdm(range(N_EPISODES), desc="Evaluation episodes"):
                obs, _ = self.env.reset(seed=seed)
                episode_rewards = []
                done = False

                while not done:
                    # Convert observation to torch tensor
                    obs_tensor = torch.tensor(obs, dtype=torch.float32).to(self.device)

                    action, _ = self.model.predict(obs_tensor, deterministic=True)
                    obs, reward, done, truncated, _ = self.env.step(action)
                    episode_rewards.append(reward)

                    if render:
                        self.env.render()

                    if truncated:
                        break

                all_rewards.append(np.sum(episode_rewards))

        return EvaluationData(
            episodes=list(range(len(all_rewards))),
            rewards=all_rewards,
        )

    @staticmethod
    def collect_episodes_rewards(model, env) -> list[float]:
        """
        Collect rewards from evaluating the agent for a given number of episodes.
        """
        rewards = []
        for seed in tqdm(SEEDS, desc="seeds"):
            for _ in tqdm(range(N_EPISODES), desc="Collect episodes"):
                obs, _ = env.reset(seed=seed)
                episode_rewards = 0
                done = False

                while not done:
                    action, _ = model.predict(obs, deterministic=True)
                    obs, reward, done, truncated, _ = env.step(action)
                    episode_rewards += reward
                rewards.append(episode_rewards)

        return rewards

    def optimize(self) -> None:
        """
        Optimize the agent with hyperparameters and return TrainingData.
        """

        def objective(trial):
            # Hyperparameter search space
            learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-3, log=True)
            n_steps = trial.suggest_int("n_steps", 32, 1024, log=True)
            batch_size = trial.suggest_int("batch_size", 16, 256, log=True)
            gamma = trial.suggest_float("gamma", 0.9, 0.9999)
            gae_lambda = trial.suggest_float("gae_lambda", 0.8, 1.0)
            ent_coef = trial.suggest_float("ent_coef", 0.0, 0.02)
            vf_coef = trial.suggest_float("vf_coef", 0.1, 1.0)
            clip_range = trial.suggest_float("clip_range", 0.1, 0.3)
            max_grad_norm = trial.suggest_float("max_grad_norm", 0.1, 1.0)

            # Set up environment and model with the sampled hyperparameters
            env = Monitor(
                ActionMasker(
                    ElectricityMarketEnv(self.env_config, render_mode="human"),
                    self.mask_fn,
                )
            )

            model = MaskablePPO(
                MaskableActorCriticPolicy,
                env,
                learning_rate=learning_rate,
                n_steps=n_steps,
                batch_size=batch_size,
                gamma=gamma,
                gae_lambda=gae_lambda,
                ent_coef=ent_coef,
                vf_coef=vf_coef,
                clip_range=clip_range,
                max_grad_norm=max_grad_norm,
                verbose=0,
                tensorboard_log=f"./{self.agent_name}_tensorboard/",
                device=self.device,
            )

            # Train and evaluate
            model.learn(
                total_timesteps=self.env_config.max_timestep,
                use_masking=True,
                reset_num_timesteps=False,
            )

            # Collect rewards for evaluation
            episode_rewards = self.collect_episodes_rewards(model, env)

            # Return the mean reward as the optimization goal
            return np.mean(episode_rewards)

        # Create a study to optimize the objective function
        study = optuna.create_study(
            study_name=self.agent_name,
            storage="sqlite:///optuna_study.db",
            load_if_exists=True,
            direction="maximize",
            pruner=optuna.pruners.HyperbandPruner(),
            sampler=optuna.samplers.TPESampler(),
        )

        # Optimize using the defined objective function
        study.optimize(objective, n_trials=N_TRAILS, n_jobs=-1, show_progress_bar=True)

        # Save the best hyperparameters
        self.optimized_hyperparameters = study.best_params

        # Re-train the model with the optimized hyperparameters
        self.model = MaskablePPO(
            MaskableActorCriticPolicy,
            self.env,
            **self.optimized_hyperparameters,
            verbose=0,
            tensorboard_log=f"./{self.agent_name}_tensorboard/",
            device=self.device,
        )

    def export_hyperparameters(self, filename: str):
        """
        Export optimized hyperparameters to a YAML file.
        """
        with open(filename, "w") as file:
            yaml.dump(self.optimized_hyperparameters, file)

    def save_model(self, model_path: Path) -> None:
        self.model.save(str(model_path))

    def load_model(self, model_path: Path) -> None:
        self.model.load(str(model_path))

### Evaluation MaskableRandom on ElectricityMarketEnv


In [ ]:
# | export
maskable_random_agent = MaskableRandomAgent(render_mode="human")

evaluation_data_per_agent[maskable_random_agent.agent_name] = (
    maskable_random_agent.evaluate()
)

seeds:   0%|          | 0/41 [00:00<?, ?it/s]

episodes:   0%|          | 0/3 [00:00<?, ?it/s]

episodes:   0%|          | 0/3 [00:00<?, ?it/s]

episodes:   0%|          | 0/3 [00:00<?, ?it/s]

episodes:   0%|          | 0/3 [00:00<?, ?it/s]

...

episodes:   0%|          | 0/3 [00:00<?, ?it/s]

episodes:   0%|          | 0/3 [00:00<?, ?it/s]

episodes:   0%|          | 0/3 [00:00<?, ?it/s]

episodes:   0%|          | 0/3 [00:00<?, ?it/s]

episodes:   0%|          | 0/3 [00:00<?, ?it/s]

### Evaluation A2C on ElectricityMarketEnv


In [ ]:
# | export
a2c_agent = A2CAgent(render_mode="human")

training_data_per_agent[a2c_agent.agent_name] = a2c_agent.train()

a2c_agent.save_model(f"{a2c_agent.agent_name}.model")

evaluation_data_per_agent[a2c_agent.agent_name] = a2c_agent.evaluate()

seeds:   0%|          | 0/41 [00:00<?, ?it/s]

Training episodes:   0%|          | 0/3 [00:00<?, ?it/s]

Training episodes:   0%|          | 0/3 [00:00<?, ?it/s]

Training episodes:   0%|          | 0/3 [00:00<?, ?it/s]

### Evaluation MaskablePPO with default hyperparameters on ElectricityMarketEnv

In [ ]:
# | export
maskable_ppo_agent = MaskablePPOAgent(render_mode="human")

training_data_per_agent[maskable_ppo_agent.agent_name] = maskable_ppo_agent.train()

maskable_ppo_agent.save_model(f"{maskable_ppo_agent.agent_name}.model")

evaluation_data_per_agent[maskable_ppo_agent.agent_name] = maskable_ppo_agent.evaluate()

### Evaluation MaskablePPO with optimized hyperparameters on ElectricityMarketEnv

In [ ]:
# | export
optimized_maskable_ppo_agent = MaskablePPOAgent(render_mode="human")
optimized_maskable_ppo_agent.agent_name = "OptimizedMaskablePPOAgent"
optimized_maskable_ppo_agent.optimize()

optimized_maskable_ppo_agent.export_hyperparameters(
    f"{optimized_maskable_ppo_agent.agent_name}.yaml"
)


training_data_per_agent[optimized_maskable_ppo_agent.agent_name] = (
    optimized_maskable_ppo_agent.train()
)

optimized_maskable_ppo_agent.save_model(
    f"{optimized_maskable_ppo_agent.agent_name}.model"
)

evaluation_data_per_agent[optimized_maskable_ppo_agent.agent_name] = (
    optimized_maskable_ppo_agent.evaluate()
)

### Adding expert knowledge to the masking function making learning more efficient

In [ ]:
# | export


def is_action_safe(self, action: int) -> bool:
    charge_amount = self._charge_amount(action)
    target_state_of_charge = self._current_state_of_charge + charge_amount
    low, high = self._battery_safe_range
    return high > target_state_of_charge > low


def expert_knowledge_action_masks(self) -> np.ndarray:
    mask = np.array(
        [
            self._is_action_valid(action) and self.is_action_safe(action)
            for action in range(self.action_space.n)
        ],
        dtype=bool,
    )
    if not np.any(mask):  # If all actions are invalid, force one to be valid
        mask[len(mask) // 2] = True
    return mask


# Dynamically overriding action_masks to ElectricityMarketEnv
setattr(ElectricityMarketEnv, "action_masks", expert_knowledge_action_masks)
# Dynamically overriding injection is_action_safe to ElectricityMarketEnv
setattr(ElectricityMarketEnv, "is_action_safe", is_action_safe)



### Evaluation MaskableRandomAgent with Expert Knowledge on ElectricityMarketEnv

In [ ]:
# | export
expert_maskable_random_agent = MaskableRandomAgent(render_mode="human")
expert_maskable_random_agent.agent_name = "ExpertMaskableRandomAgent"

evaluation_data_per_agent[expert_maskable_random_agent.agent_name] = (
    expert_maskable_random_agent.evaluate()
)

### Evaluation MaskablePPO with default hyperparameters and Expert Knowledge on ElectricityMarketEnv

In [ ]:
# | export
expert_maskable_ppo_agent = MaskablePPOAgent(render_mode="human")
expert_maskable_ppo_agent.agent_name = "ExpertMaskablePPOAgent"

training_data_per_agent[expert_maskable_ppo_agent.agent_name] = (
    expert_maskable_ppo_agent.train()
)

expert_maskable_ppo_agent.save_model(f"{expert_maskable_ppo_agent.agent_name}.model")

evaluation_data_per_agent[expert_maskable_ppo_agent.agent_name] = (
    expert_maskable_ppo_agent.evaluate()
)

### Evaluation MaskablePPO with optimized hyperparameters and Expert Knowledge on ElectricityMarketEnv


In [ ]:
# | export
optimized_expert_maskable_ppo_agent = MaskablePPOAgent(render_mode="human")
optimized_expert_maskable_ppo_agent.agent_name = "OptimizedExpertMaskablePPOAgent"
optimized_expert_maskable_ppo_agent.optimize()

optimized_expert_maskable_ppo_agent.export_hyperparameters(
    f"{optimized_expert_maskable_ppo_agent.agent_name}.yaml"
)

training_data_per_agent[optimized_expert_maskable_ppo_agent.agent_name] = (
    optimized_expert_maskable_ppo_agent.train()
)

optimized_expert_maskable_ppo_agent.save_model(
    f"{optimized_expert_maskable_ppo_agent.agent_name}.model"
)

evaluation_data_per_agent[optimized_expert_maskable_ppo_agent.agent_name] = (
    optimized_expert_maskable_ppo_agent.evaluate()
)

In [ ]:
# | export
plot_all_metrics(training_data_per_agent, evaluation_data_per_agent)

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()